Download the dataset from the link provided using !wget command

In [1]:
# !wget -q https://datasetsgun.s3.amazonaws.com/upgrad/flowers-20210317T060850Z-001.zip

/bin/sh: 1: wget: not found


Unzip the downloaded file using !unzip command

In [ ]:
# !unzip -q flowers-20210317T060850Z-001.zip

Create a new directory 'data' to save your Train & Test data

In [1]:
import os
os.mkdir('data')

In [2]:
import os
import shutil
import numpy as np
import glob   
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import  plot_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline


['tensorflow-tutorials', 'data', '.ipynb_checkpoints']

In [3]:
# Where all dataset is there
data_dir = 'flowers/'

# Training data dir
training_dir = 'data/Train'

# Test data dir
testing_dir = 'data/Test'

# Ratio of training and testing data
train_test_ratio = 0.8 


def split_dataset_into_test_and_train_sets(all_data_dir = data_dir, training_data_dir = training_dir, testing_data_dir=testing_dir, train_test_ratio = 0.8):
    # Recreate testing and training directories
    
    if not os.path.exists(training_data_dir):
            os.mkdir(training_data_dir)

    if not os.path.exists(testing_data_dir):
            os.mkdir(testing_data_dir)               
    
    num_training_files = 0
    num_testing_files = 0


    for subdir, dirs, files in os.walk(all_data_dir):
        
        category_name = os.path.basename(subdir)
        
        # print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
          continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name
        
        # creating subdir for each sub category
        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)   

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)
            
        file_list = glob.glob(os.path.join(subdir,'*.jpg'))

        #print(os.path.join(all_data_dir, subdir))
        print(str(category_name) + ' has ' + str(len(files)) + ' images') 
        random_set = np.random.permutation((file_list))
        # copy percentage of data from each category to train and test directory
        train_list = random_set[:round(len(random_set)*(train_test_ratio))] 
        test_list = random_set[-round(len(random_set)*(1-train_test_ratio)):]

  

        for lists in train_list : 
            shutil.copy(lists, training_data_dir + '/' + category_name + '/' )
            num_training_files += 1
  
        for lists in test_list : 
            shutil.copy(lists, testing_data_dir + '/' + category_name + '/' )
            num_testing_files += 1
  

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [4]:
split_dataset_into_test_and_train_sets()

daisy has 778 images
dandelion has 1055 images
data has 0 images
flowers has 0 images
daisy has 769 images
dandelion has 1055 images
rose has 784 images
sunflower has 734 images
tulip has 984 images
rose has 784 images
sunflower has 734 images
tulip has 984 images
Processed 6923 training files.
Processed 1732 testing files.


In [6]:
# Number of classes in dataset
num_classes = 5

def get_model():
    # Get base model 
    # Here we are using ResNet50 as base model
    base_model = ResNet50(weights='imagenet', include_top=False)
    
    # As we are using ResNet model only for feature extraction and not adjusting the weights
    # we freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = False
        
    # Get base model output 
    base_model_ouput = base_model.output
    
    # Adding our own layer 
    x = GlobalAveragePooling2D()(base_model_ouput)
    # Adding fully connected layer
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [7]:
# Get the model
model = get_model()
# Compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Summary of model
model.summary()

94773248/94765736 [==============================] - 12s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________

In [12]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64

train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

Found 4151 images belonging to 5 classes.
Found 1555 images belonging to 5 classes.


In [13]:
# Training the fully conncected layer for initial epochs
epochs = 5

# Training the model

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

Epoch 1/5
64/64 [==============================] - 46s 710ms/step - loss: 0.7001 - accuracy: 0.7592 - val_loss: 0.4164 - val_accuracy: 0.8698
Epoch 2/5
64/64 [==============================] - 46s 715ms/step - loss: 0.3956 - accuracy: 0.8657 - val_loss: 0.3568 - val_accuracy: 0.8809
Epoch 3/5
64/64 [==============================] - 47s 730ms/step - loss: 0.3352 - accuracy: 0.8899 - val_loss: 0.2969 - val_accuracy: 0.9017
Epoch 4/5
64/64 [==============================] - 48s 741ms/step - loss: 0.2997 - accuracy: 0.9009 - val_loss: 0.2671 - val_accuracy: 0.9115
Epoch 5/5
64/64 [==============================] - 48s 744ms/step - loss: 0.2691 - accuracy: 0.9078 - val_loss: 0.2405 - val_accuracy: 0.9193


In [14]:
# More fine tuning the model
# Training the model after 150 layers
# Generally ResNet is good at extracting lower level features so we are not fine tuning initial layers
epochs = 10

split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
    
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate, if we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

d:\Repositories\Upgrad-PGD-ML---AI\tensorflow_env\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
64/64 [==============================] - 55s 779ms/step - loss: 0.3846 - accuracy: 0.8898 - val_loss: 0.2174 - val_accuracy: 0.9206
Epoch 2/10
64/64 [==============================] - 47s 729ms/step - loss: 0.2396 - accuracy: 0.9250 - val_loss: 0.1506 - val_accuracy: 0.9479
Epoch 3/10
64/64 [==============================] - 49s 759ms/step - loss: 0.1737 - accuracy: 0.9485 - val_loss: 0.1235 - val_accuracy: 0.9590
Epoch 4/10
64/64 [==============================] - 48s 749ms/step - loss: 0.1325 - accuracy: 0.9660 - val_loss: 0.0960 - val_accuracy: 0.9694
Epoch 5/10
64/64 [==============================] - 52s 804ms/step - loss: 0.1159 - accuracy: 0.9635 - val_loss: 0.0864 - val_accuracy: 0.9779
Epoch 6/10
64/64 [==============================] - 49s 766ms/step - loss: 0.0961 - accuracy: 0.9763 - val_loss: 0.0634 - val_accuracy: 0.9863
Epoch 7/10
64/64 [==============================] - 49s 762ms/step - loss: 0.0738 - accuracy: 0.9799 - val_loss: 0.0596 - val_accuracy: 0.9870

In [15]:
print('Training complete')

Training complete
